In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import time
import os
import pickle
from torch.utils.data import TensorDataset, DataLoader  
from modules.utils import load_cv, norm_data, get_class_weights
from modules.eval_funcs import cv_fold_eval
from modules.cleaning import rr_prefix
from modules.models import FeedFwdNN, MlpModel
from modules.lr_scheduler import build_scheduler
from modules.tab_train import train_IBP_model, EarlyStopper, test_eval, pycyto_norm
from torch.optim.lr_scheduler import CosineAnnealingLR
from timm.scheduler.cosine_lr import CosineLRScheduler

rand_seed = 42
moa_dict = {'PI3K' : 0, 'p38 MAPK': 1, 'RAF': 2, 'AURK': 3, 'CDK': 4, 'EGFR': 5, 'ROCK': 6,
             'MEK': 7, 'GSK': 8, 'mTOR': 9}

# Load Data:

In [2]:
ki_ibp = pd.read_csv('data/ibp/ki_ibp.csv')
print(ki_ibp.shape)
ki_ibp.head(2)

(635, 4778)


,Metadata_Source,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,...,smiles,clinical_phase,moa_src,Metadata_JCP2022,Metadata_InChIKey,Metadata_PlateType,blur_score,sat_score,focus_score,comp_score
0,source_3,JCPQC023,G14,3227.817708,5310.328125,589.21875,541.552083,519.484375,471.942708,553.446757,...,Nc1cc(c(cn1)-c1cc(nc(n1)N1CCOCC1)N1CCOCC1)C(F)...,Phase 3,dr_hub,JCP2022_013856,CWHUFRVAEUJCEF-UHFFFAOYSA-N,TARGET2,0.430742,0.453621,0.517562,1.401925
1,source_4,BR00121424,G14,4255.300000,7338.300000,572.70000,554.500000,488.420000,470.490000,530.260000,...,Nc1cc(c(cn1)-c1cc(nc(n1)N1CCOCC1)N1CCOCC1)C(F)...,Phase 3,dr_hub,JCP2022_013856,CWHUFRVAEUJCEF-UHFFFAOYSA-N,TARGET2,0.436727,0.144924,0.386009,0.967661


In [5]:
chem_keys = ki_ibp.Metadata_InChIKey.unique()

In [6]:
# Specify the file path
file_path = "chem_keys.txt"

# Open the file in write mode and write each string on a new line
with open(file_path, 'w') as file:
    for string in chem_keys:
        file.write(string + '\n')

# Load CV Fold Data:

In [3]:
# Load CV split data:
cv_splits = 5
cv_path = 'data/cv_val_split/'
cv_data = load_cv(cv_path, cv_splits, ki_ibp, moa_dict, norm=None)

In [7]:
# Load Spherized and Harmonized Data:
pycy_spher = 'output/MAD_Sphere/MADS_PyCyFS.pkl'
shap_spher = 'output/MAD_Sphere/MADS_ShapFS.pkl'
pycy_harm = 'output/MAD_Harmony/MAD_Harmony_PyCyFS.pkl'
shap_harm = 'output/MAD_Harmony/MAD_Harmony_ShapFS.pkl'

with open(shap_spher, 'rb') as file:
    loaded_data = pickle.load(file)
    
# Specify the prefix to remove
prefix_to_remove = 'sph_'
loaded_data = rr_prefix(loaded_data, prefix_to_remove)

In [5]:
# Load Feature Selection Data:
cv_results = pd.read_csv('output/feat_sel/shap_fs_res.csv')

In [8]:
for i in range(5):
    train_meta = loaded_data[1]['train_meta'].reset_index(drop=True)
    test_meta = loaded_data[1]['test_meta'].reset_index(drop=True)
    uniq_c = len(train_meta.Metadata_JCP2022.unique()) + len(test_meta.Metadata_JCP2022.unique())
    print(f"CV{i} Unique Compounds retained: ", uniq_c)

CV0 Unique Compounds retained:  93
CV1 Unique Compounds retained:  93
CV2 Unique Compounds retained:  93
CV3 Unique Compounds retained:  93
CV4 Unique Compounds retained:  93


In [10]:
shap_spher = 'output/Corrected/Sph_Shap_Cor.pkl'

with open(shap_spher, 'rb') as file:
    loaded_data = pickle.load(file)

# Specify the prefix to remove
prefix_to_remove = 'sph_'
loaded_data = rr_prefix(loaded_data, prefix_to_remove)

In [5]:
for i in range(5):
    train_meta = loaded_data[1]['train_meta'].reset_index(drop=True)
    test_meta = loaded_data[1]['test_meta'].reset_index(drop=True)
    uniq_c = len(train_meta.Metadata_JCP2022.unique()) + len(test_meta.Metadata_JCP2022.unique())
    print(f"CV{i} Unique Compounds retained: ", uniq_c)

CV0 Unique Compounds retained:  96
CV1 Unique Compounds retained:  96
CV2 Unique Compounds retained:  96
CV3 Unique Compounds retained:  96
CV4 Unique Compounds retained:  96


In [6]:
loaded_data[1]['X_train'].head()

,Nuclei_AreaShape_Zernike_8_8,Nuclei_AreaShape_Zernike_4_4,Nuclei_AreaShape_Zernike_2_2,Nuclei_AreaShape_Zernike_8_6,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_mito_skel,Nuclei_Neighbors_FirstClosestDistance_1,Nuclei_AreaShape_Eccentricity,Cells_Neighbors_PercentTouching_Adjacent,Cytoplasm_Texture_Correlation_Mito_10_02_256,Nuclei_Texture_Correlation_RNA_10_02_256,...,Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_14of16,Nuclei_AreaShape_Zernike_4_0,Nuclei_Correlation_Correlation_ER_Mito,Cells_RadialDistribution_RadialCV_mito_tubeness_1of16,Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_9of16,Image_Granularity_13_AGP,Cells_RadialDistribution_MeanFrac_DNA_2of4,Image_Granularity_13_Mito,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_RadialDistribution_FracAtD_ER_3of4
0,-0.023415,0.027976,0.048090,0.030426,0.101084,-0.446435,0.014585,-0.165405,0.205840,0.278134,...,0.012394,-0.040926,-0.039585,-0.191058,0.078121,1.801455,-0.060201,-0.613835,-0.668586,-0.017622
1,-0.212893,0.315910,0.361765,-0.068824,-0.017317,-1.284632,0.276249,-3.204976,-0.077054,-0.098465,...,0.253602,-0.179778,-0.640755,-1.236269,-0.039896,1.265960,-0.847406,2.455292,-4.242798,0.208505
2,0.124266,0.485932,0.128665,-0.446882,-0.197252,-2.008731,-0.002011,-0.590232,1.133575,0.073882,...,-0.126733,-0.085436,-0.783395,-1.307992,-0.005549,0.683855,-1.656494,0.876375,-3.339853,-0.083263
3,-0.128326,-0.027578,0.105256,0.069811,0.055336,0.234117,0.145965,-0.436490,-0.172529,0.279556,...,-0.111030,-0.044801,0.173719,-0.442135,0.045679,6.099989,0.056262,0.781920,-0.547906,-0.061829
4,-0.437595,0.997057,1.887933,0.743784,-1.491994,2.612060,0.015014,-1.361556,-0.482907,-0.665730,...,-0.840691,-1.258644,-0.433420,1.226237,-0.651601,-1.047995,0.327151,4.361536,14.416534,0.134484


# Kaggle FFNN Model:
Implementing Pytorch, multiclass, single-input version of the Kaggle-winning MOA prediction model, code available at:

https://github.com/guitarmind/kaggle_moa_winner_hungry_for_gold/blob/main/final//Best%20LB/Training/2heads-ResNest-train.ipynb

**Essential Model Characteristics:**
- 6 fully connected layers with batch-norm.
- Drop out in first two hidden layers.
- Combinations of 'elu' (first, third), 'relu' (second, third), and 'selu' activation functions (fifth layer).
- Adam optimizer.
- Reduce lr on plateau by a factor of 0.5 with patience of 4.

**Model imported from modules.models**

## Static Variables:

In [43]:
# Model parameters:
num_classes = 10
n_epochs = 95
batch_size = 16
n_neurons = 2048
lr = 1e-5
device = 'cpu'
sched_type = 'linear'
run_dir = 'output/IBP_MLP/MADH_SHAP_FINAL/'
if not os.path.exists(run_dir):
        os.makedirs(run_dir)
early_stop = 95
opt_wd = 1e-8
norm = True
dropout = 0.2
eq_weights = False
full_train = False
tab_norm = 'mads'

# Shapely Features:
n_features = loaded_data[0]['X_train'].shape[1] # 150 #len(cv_data[0]['X_train'].columns)
top_feats = cv_results['features'][0:n_features].tolist()

In [44]:
class Args:
    def __init__(self, num_classes, n_epochs, batch_size, n_neurons, lr, sched_type, 
                 n_features, opt_wd, norm, dropout, eq_weights, full_train, tab_norm):
        self.num_classes = num_classes
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.n_neurons = n_neurons
        self.lr = lr
        self.sched_type = sched_type
        self.t_initial = 8
        self.cycle_mul = 2
        self.min_lr = 5e-6
        self.lrd_fac = 0.5
        self.warmup_epochs = 10
        self.warmup_lr = 5e-6
        self.lr_restarts = 5
        self.lr_pat = 6
        self.workers = 1
        self.model_type = 'FFNN'
        self.n_features = n_features
        self.opt_wd = opt_wd
        self.norm = norm
        self.dropout = dropout
        self.eq_weights = eq_weights
        self.full_train = full_train
        self.tab_norm = tab_norm

args = Args(num_classes, n_epochs, batch_size, n_neurons, lr, sched_type, 
            n_features, opt_wd, norm, dropout, eq_weights, full_train, tab_norm)

with open('{}/args_vars.txt'.format(run_dir), 'w') as file:
    for key, value in vars(args).items():
        file.write(f"{key}: {value}\n")

# Model Training Loop:

In [45]:
cv_best_mods = []
acc_scores = []
f1_scores = []
res_dict = {}

# Lists to hold best results from CV folds:
cv_train_loss = []
cv_train_acc = []
cv_val_loss = []
cv_val_acc = []
cv_cpnd_acc = []

for i, fold in enumerate(loaded_data):
    best_cv_mod = 0
    
    # Create folder for training output:
    train_dir = os.path.join(run_dir, 'CV%s_Output/' % i)
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    
    # DEFINE MODEL, OPTIMIZER, CRITERION AND SCHEDULER:
    args.n_features = loaded_data[i]['X_train'].shape[1] # variable number of features selected by pycy per fold
    model = FeedFwdNN(args.n_features, args.num_classes, args.n_neurons, args.dropout)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=args.opt_wd)
    # Equal Weight Loss Function
    if args.eq_weights:  
        class_weights = get_class_weights(fold['y_train'])
        criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        criterion = nn.CrossEntropyLoss()
        
    lr_scheduler = build_scheduler(args, optimizer)
    early_stopper = EarlyStopper(patience=early_stop, min_delta=1e-5)
    
    # Extract cv-fold training and validation sets:
    if tab_norm == 'minmax':
        cv_X_train = fold['X_train'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_X_val = fold['X_val'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_X_test = fold['X_test'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_y_train = fold['y_train']
        cv_y_val = fold['y_val']
        cv_y_test = fold['y_test']
        val_meta = fold['val_meta'].Metadata_JCP2022.to_list()
        test_meta = fold['test_meta'].Metadata_JCP2022.to_list()

        # Combine training and validation sets if using full training set:
        if args.full_train:
            cv_X_train = pd.concat([cv_X_train, cv_X_val]).reset_index(drop=True)
            cv_y_train = np.concatenate((cv_y_train, cv_y_val))       
    
        # Normalize Feature Data:
        if args.norm:
            cv_X_train, cv_X_val, cv_X_test = norm_data('minmax', cv_X_train, cv_X_val, cv_X_test)
        
        # Train Model:
        vl_mod, va_mod, mdl_list = train_IBP_model(i, cv_X_train, cv_X_val, cv_y_train, cv_y_val, val_meta,
                                     model, optimizer, criterion, device, train_dir, args,
                                     lr_scheduler, early_stopper)
    
    elif tab_norm == 'mads':
        cv_X_train = loaded_data[i]['X_train'].astype(np.float32)
        cv_X_test = loaded_data[i]['X_test'].astype(np.float32)
        cv_y_train = loaded_data[i]['y_train'].values
        cv_y_test = loaded_data[i]['y_test'].values
        test_meta = loaded_data[i]['test_meta'].Metadata_JCP2022.to_list()
       
        # Train Model:
        vl_mod, va_mod, mdl_list = train_IBP_model(i, cv_X_train, cv_X_test, cv_y_train, cv_y_test, test_meta,
                                     model, optimizer, criterion, device, train_dir, args,
                                     lr_scheduler, early_stopper)
    
    
    # TEST SET EVALUATION:
    print("___________TEST EVALUATION - CV %s____________" % i)
#     print("--------- VLoss Model Metrics ---------")
#     acc_VL, f1_VL, _ = test_eval(i, vl_mod, cv_X_test, cv_y_test, test_meta, device, 
#                                  train_dir, 'FFNN')
#     print("--------- VAcc Model Metrics ---------")
#     acc_VA, f1_VA, _ = test_eval(i, va_mod, cv_X_test, cv_y_test, test_meta, device, 
#                                  train_dir, 'FFNN')
    print("--------- Final Model Metrics ---------")
    acc_end, f1_end, _ = test_eval(i, mdl_list[-1], cv_X_test, cv_y_test, test_meta, device, 
                                   train_dir, 'FFNN')
    print("---------------------------------------\n")
    
    acc_scores.append(acc_end) #max(acc_VL, acc_VA))
    f1_scores.append(f1_end) #max(f1_VL, f1_VA))
    
#     if i >= 0:
#         break
    
# Updating Results Dictionary:
res_dict['CV Accuracy'] = acc_scores
res_dict['CV F1 Score'] = f1_scores
res_dict['Average Accuracy'] = np.mean(acc_scores)
res_dict['Average F1 Score'] = np.mean(f1_scores)
    
# Save Results:    
with open('{}/results_{:.2f}_Acc.txt'.format(run_dir, res_dict['Average Accuracy']), 'w') as file:
    for key, value in res_dict.items():
        file.write(f"{key}: {value}\n")

Epoch 1/95: Train Loss: 2.241 | Train Well Acc: 16.07 |Val Loss: 2.139 | Val Well Acc:15.62 | Val Cpnd Acc:15.00 | Time Taken:8s
Epoch 2/95: Train Loss: 2.069 | Train Well Acc: 22.54 |Val Loss: 2.051 | Val Well Acc:25.78 | Val Cpnd Acc:35.00 | Time Taken:7s
Epoch 3/95: Train Loss: 1.958 | Train Well Acc: 31.25 |Val Loss: 1.957 | Val Well Acc:32.81 | Val Cpnd Acc:40.00 | Time Taken:7s
Epoch 4/95: Train Loss: 1.866 | Train Well Acc: 34.60 |Val Loss: 1.920 | Val Well Acc:32.81 | Val Cpnd Acc:40.00 | Time Taken:7s
Epoch 5/95: Train Loss: 1.786 | Train Well Acc: 38.84 |Val Loss: 1.902 | Val Well Acc:35.16 | Val Cpnd Acc:45.00 | Time Taken:7s
Epoch 6/95: Train Loss: 1.741 | Train Well Acc: 40.85 |Val Loss: 1.875 | Val Well Acc:35.94 | Val Cpnd Acc:45.00 | Time Taken:7s
Epoch 7/95: Train Loss: 1.691 | Train Well Acc: 46.21 |Val Loss: 1.839 | Val Well Acc:38.28 | Val Cpnd Acc:55.00 | Time Taken:7s
Epoch 8/95: Train Loss: 1.696 | Train Well Acc: 43.75 |Val Loss: 1.832 | Val Well Acc:37.50 | Val

Epoch 65/95: Train Loss: 1.187 | Train Well Acc: 63.17 |Val Loss: 1.684 | Val Well Acc:48.70 | Val Cpnd Acc:60.00 | Time Taken:6s
Epoch 66/95: Train Loss: 1.157 | Train Well Acc: 60.49 |Val Loss: 1.741 | Val Well Acc:46.35 | Val Cpnd Acc:65.00 | Time Taken:7s
Epoch 67/95: Train Loss: 1.195 | Train Well Acc: 58.48 |Val Loss: 1.695 | Val Well Acc:52.86 | Val Cpnd Acc:65.00 | Time Taken:6s
Epoch 68/95: Train Loss: 1.173 | Train Well Acc: 62.05 |Val Loss: 1.698 | Val Well Acc:48.70 | Val Cpnd Acc:65.00 | Time Taken:7s
Epoch 69/95: Train Loss: 1.157 | Train Well Acc: 61.83 |Val Loss: 1.690 | Val Well Acc:48.70 | Val Cpnd Acc:60.00 | Time Taken:7s
Epoch 70/95: Train Loss: 1.168 | Train Well Acc: 62.50 |Val Loss: 1.699 | Val Well Acc:50.52 | Val Cpnd Acc:60.00 | Time Taken:7s
Epoch 71/95: Train Loss: 1.177 | Train Well Acc: 61.83 |Val Loss: 1.716 | Val Well Acc:47.14 | Val Cpnd Acc:65.00 | Time Taken:8s
Epoch 72/95: Train Loss: 1.171 | Train Well Acc: 59.38 |Val Loss: 1.698 | Val Well Acc:46.

Epoch 31/95: Train Loss: 1.376 | Train Well Acc: 54.02 |Val Loss: 2.030 | Val Well Acc:36.20 | Val Cpnd Acc:61.11 | Time Taken:7s
Epoch 32/95: Train Loss: 1.374 | Train Well Acc: 55.36 |Val Loss: 1.993 | Val Well Acc:36.20 | Val Cpnd Acc:55.56 | Time Taken:7s
Epoch 33/95: Train Loss: 1.341 | Train Well Acc: 54.91 |Val Loss: 2.004 | Val Well Acc:37.76 | Val Cpnd Acc:55.56 | Time Taken:7s
Epoch 34/95: Train Loss: 1.303 | Train Well Acc: 59.82 |Val Loss: 1.989 | Val Well Acc:40.10 | Val Cpnd Acc:50.00 | Time Taken:6s
Epoch 35/95: Train Loss: 1.340 | Train Well Acc: 54.46 |Val Loss: 2.028 | Val Well Acc:37.76 | Val Cpnd Acc:50.00 | Time Taken:6s
Epoch 36/95: Train Loss: 1.283 | Train Well Acc: 57.81 |Val Loss: 2.038 | Val Well Acc:37.76 | Val Cpnd Acc:50.00 | Time Taken:7s
Epoch 37/95: Train Loss: 1.339 | Train Well Acc: 54.46 |Val Loss: 2.018 | Val Well Acc:37.76 | Val Cpnd Acc:55.56 | Time Taken:7s
Epoch 38/95: Train Loss: 1.269 | Train Well Acc: 57.81 |Val Loss: 2.005 | Val Well Acc:37.

Epoch 95/95: Train Loss: 1.108 | Train Well Acc: 61.83 |Val Loss: 2.073 | Val Well Acc:37.50 | Val Cpnd Acc:50.00 | Time Taken:7s
-----> CV-FOLD 2 Training Complete: | Lowest Val Loss: 1.967 @ Epoch 23 | Max. Val Acc:40.885 @ Epoch 17:
___________TEST EVALUATION - CV 1____________
--------- Final Model Metrics ---------
Cpnd-level Accuracy: 50.00%
Cpnd-level f1 score: 49.00%
---------------------------------------

Epoch 1/95: Train Loss: 2.324 | Train Well Acc: 14.44 |Val Loss: 2.252 | Val Well Acc:23.57 | Val Cpnd Acc:27.78 | Time Taken:8s
Epoch 2/95: Train Loss: 2.016 | Train Well Acc: 30.17 |Val Loss: 2.118 | Val Well Acc:30.71 | Val Cpnd Acc:44.44 | Time Taken:7s
Epoch 3/95: Train Loss: 1.890 | Train Well Acc: 34.48 |Val Loss: 2.073 | Val Well Acc:34.29 | Val Cpnd Acc:44.44 | Time Taken:7s
Epoch 4/95: Train Loss: 1.811 | Train Well Acc: 38.79 |Val Loss: 2.026 | Val Well Acc:42.50 | Val Cpnd Acc:55.56 | Time Taken:7s
Epoch 5/95: Train Loss: 1.766 | Train Well Acc: 39.01 |Val Loss: 

Epoch 61/95: Train Loss: 1.170 | Train Well Acc: 62.50 |Val Loss: 1.825 | Val Well Acc:44.29 | Val Cpnd Acc:66.67 | Time Taken:7s
Epoch 62/95: Train Loss: 1.236 | Train Well Acc: 58.62 |Val Loss: 1.803 | Val Well Acc:46.07 | Val Cpnd Acc:66.67 | Time Taken:7s
Epoch 63/95: Train Loss: 1.171 | Train Well Acc: 64.01 |Val Loss: 1.831 | Val Well Acc:46.96 | Val Cpnd Acc:66.67 | Time Taken:7s
Epoch 64/95: Train Loss: 1.167 | Train Well Acc: 63.58 |Val Loss: 1.858 | Val Well Acc:47.86 | Val Cpnd Acc:66.67 | Time Taken:7s
Epoch 65/95: Train Loss: 1.214 | Train Well Acc: 60.34 |Val Loss: 1.823 | Val Well Acc:47.86 | Val Cpnd Acc:66.67 | Time Taken:8s
Epoch 66/95: Train Loss: 1.194 | Train Well Acc: 62.72 |Val Loss: 1.789 | Val Well Acc:47.86 | Val Cpnd Acc:61.11 | Time Taken:8s
Epoch 67/95: Train Loss: 1.210 | Train Well Acc: 60.78 |Val Loss: 1.819 | Val Well Acc:45.18 | Val Cpnd Acc:66.67 | Time Taken:8s
Epoch 68/95: Train Loss: 1.235 | Train Well Acc: 57.76 |Val Loss: 1.809 | Val Well Acc:47.

Epoch 27/95: Train Loss: 1.344 | Train Well Acc: 56.25 |Val Loss: 1.802 | Val Well Acc:39.84 | Val Cpnd Acc:50.00 | Time Taken:7s
Epoch 28/95: Train Loss: 1.356 | Train Well Acc: 51.56 |Val Loss: 1.809 | Val Well Acc:42.97 | Val Cpnd Acc:55.56 | Time Taken:7s
Epoch 29/95: Train Loss: 1.360 | Train Well Acc: 56.25 |Val Loss: 1.795 | Val Well Acc:43.75 | Val Cpnd Acc:55.56 | Time Taken:6s
Epoch 30/95: Train Loss: 1.330 | Train Well Acc: 56.92 |Val Loss: 1.810 | Val Well Acc:46.88 | Val Cpnd Acc:61.11 | Time Taken:7s
Epoch 31/95: Train Loss: 1.267 | Train Well Acc: 60.04 |Val Loss: 1.796 | Val Well Acc:42.97 | Val Cpnd Acc:61.11 | Time Taken:8s
Epoch 32/95: Train Loss: 1.295 | Train Well Acc: 60.49 |Val Loss: 1.769 | Val Well Acc:42.19 | Val Cpnd Acc:61.11 | Time Taken:6s
Epoch 33/95: Train Loss: 1.341 | Train Well Acc: 56.47 |Val Loss: 1.786 | Val Well Acc:41.41 | Val Cpnd Acc:61.11 | Time Taken:6s
Epoch 34/95: Train Loss: 1.280 | Train Well Acc: 59.82 |Val Loss: 1.819 | Val Well Acc:39.

Epoch 91/95: Train Loss: 1.050 | Train Well Acc: 65.40 |Val Loss: 1.980 | Val Well Acc:38.28 | Val Cpnd Acc:50.00 | Time Taken:7s
Epoch 92/95: Train Loss: 1.111 | Train Well Acc: 66.07 |Val Loss: 1.939 | Val Well Acc:39.06 | Val Cpnd Acc:50.00 | Time Taken:7s
Epoch 93/95: Train Loss: 1.074 | Train Well Acc: 64.29 |Val Loss: 1.942 | Val Well Acc:39.06 | Val Cpnd Acc:50.00 | Time Taken:6s
Epoch 94/95: Train Loss: 1.072 | Train Well Acc: 63.84 |Val Loss: 1.942 | Val Well Acc:39.84 | Val Cpnd Acc:50.00 | Time Taken:6s
Epoch 95/95: Train Loss: 1.051 | Train Well Acc: 65.40 |Val Loss: 1.964 | Val Well Acc:38.28 | Val Cpnd Acc:50.00 | Time Taken:7s
-----> CV-FOLD 4 Training Complete: | Lowest Val Loss: 1.738 @ Epoch 22 | Max. Val Acc:46.875 @ Epoch 30:
___________TEST EVALUATION - CV 3____________
--------- Final Model Metrics ---------
Cpnd-level Accuracy: 55.56%
Cpnd-level f1 score: 50.71%
---------------------------------------

Epoch 1/95: Train Loss: 2.210 | Train Well Acc: 19.42 |Val Lo

Epoch 57/95: Train Loss: 1.214 | Train Well Acc: 59.82 |Val Loss: 1.862 | Val Well Acc:33.15 | Val Cpnd Acc:47.37 | Time Taken:6s
Epoch 58/95: Train Loss: 1.164 | Train Well Acc: 62.28 |Val Loss: 1.917 | Val Well Acc:31.70 | Val Cpnd Acc:52.63 | Time Taken:7s
Epoch 59/95: Train Loss: 1.213 | Train Well Acc: 58.04 |Val Loss: 1.889 | Val Well Acc:32.25 | Val Cpnd Acc:52.63 | Time Taken:7s
Epoch 60/95: Train Loss: 1.198 | Train Well Acc: 59.60 |Val Loss: 1.903 | Val Well Acc:33.15 | Val Cpnd Acc:47.37 | Time Taken:6s
Epoch 61/95: Train Loss: 1.284 | Train Well Acc: 55.36 |Val Loss: 1.881 | Val Well Acc:32.25 | Val Cpnd Acc:42.11 | Time Taken:7s
Epoch 62/95: Train Loss: 1.298 | Train Well Acc: 54.46 |Val Loss: 1.893 | Val Well Acc:34.71 | Val Cpnd Acc:47.37 | Time Taken:7s
Epoch 63/95: Train Loss: 1.168 | Train Well Acc: 59.38 |Val Loss: 1.880 | Val Well Acc:35.60 | Val Cpnd Acc:57.89 | Time Taken:7s
Epoch 64/95: Train Loss: 1.184 | Train Well Acc: 60.71 |Val Loss: 1.927 | Val Well Acc:32.

In [ ]:
eval_acc = []
    eval_f1s = []
    for mdl in mdl_list:
        print("___________TEST EVALUATION - CV %s____________" % i)
    #     print("--------- VLoss Model Metrics ---------")
    #     acc_VL, f1_VL, _ = test_eval(i, vl_mod, cv_X_test, cv_y_test, test_meta, device, 
    #                                  train_dir, 'FFNN')
    #     print("--------- VAcc Model Metrics ---------")
    #     acc_VA, f1_VA, _ = test_eval(i, va_mod, cv_X_test, cv_y_test, test_meta, device, 
    #                                  train_dir, 'FFNN')
        print("--------- Final Model Metrics ---------")
        acc_end, f1_end, _ = test_eval(i, mdl, cv_X_test, cv_y_test, test_meta, device, 
                                       train_dir, 'FFNN')
        eval_acc.append(acc_end)
        eval_f1s.append(f1_end)
        print("---------------------------------------\n")
    
    acc_scores.append(max(eval_acc))
    f1_scores.append(max(eval_f1s))